<a href="https://colab.research.google.com/github/stamkiya/cs411-data-modeling/blob/master/hw03/Assignment03Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
from time import time
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from matplotlib import pyplot as plt
from google.colab import files
import io

%matplotlib inline
plt.rcParams['figure.figsize'] = (20,15) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')


# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [17]:
from google.colab import files
uploaded = files.upload()

Saving movies_metadata.csv to movies_metadata (1).csv


In [18]:
movies= pd.read_csv(io.BytesIO(uploaded['movies_metadata.csv']))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
def euclidean(list_A, list_B):
  diff_vect = list_A - list_B
  return np.sqrt(np.dot(diff_vect,diff_vect))


def manhattan(list_A, list_B):
  # list_length_check(list_A,list_B)
  from numpy import sqrt
  a_b = zip(list_A, list_B)
  manhattan = 0
  for x,y in a_b:
      manhattan += abs(x-y)
  return manhattan


def RangeQuery(X, distance_metric, point, eps):
  N = []
  dist = 100
  for i in range(len(X)):
      if distance_metric is 'euclidean':
        dist = euclidean(X[i],point)
      if distance_metric is 'manhattan':
        dist = manhattan(X[i],point)
      if dist <= eps:
        N.append(i)
  return N

In [ ]:
def euclidean(list_A, list_B):
  diff_vect = list_A - list_B
  return np.sqrt(np.dot(diff_vect,diff_vect))


def manhattan(list_A, list_B):
  # list_length_check(list_A,list_B)
  from numpy import sqrt
  a_b = zip(list_A, list_B)
  manhattan = 0
  for x,y in a_b:
      manhattan += abs(x-y)
  return manhattan


def distance_mattrix(X, distance_metric, point, eps):
  N = []
  dist = 100
  for i in range(len(X)):
      if distance_metric is 'euclidean':
        dist = euclidean(X[i],point)
      if distance_metric is 'manhattan':
        dist = manhattan(X[i],point)
      if dist <= eps:
        N.append(i)
  return N

In [72]:
class DBSCAN():
    def fit(self, X, eps, min_samples, distance_metric):
        C=0
        label = dict.fromkeys(range(len(X)), "unvisited")
        for i in range(len(X)):
          if (label[i] == "unvisited"):
            N = RangeQuery(X, distance_metric, X[i], eps)
            if (len(N)<min_samples):
              label[i] = -1
            C = C+1
            label[i]= C
            N.remove(i)
            S = N
            for Q in range(len(S)):
                if label[S[Q]] == -1:
                  label[S[Q]] = C
                if label[S[Q]] == "unvisited":
                  label[S[Q]] = C
                Nh = RangeQuery(X, distance_metric, X[Q], eps)
                if len(Nh) >= min_samples:
                    S.append(Nh)
        return label

    def __init__(self):
            pass
            

In [73]:

#retain numeric columns for clustering
movies = movies[['budget','popularity','revenue','runtime','vote_average','vote_count']]

# drop rows with null or NaN values
movies = movies[~movies.isnull().any(axis=1)]
movies = movies.dropna()

# remove movies with less than 30 votes
movies = movies[movies['vote_count'] > 30]
movies.head()

,budget,popularity,revenue,runtime,vote_average,vote_count
0,30000000,21.9469,373554033.0,81.0,7.7,5415.0
1,65000000,17.0155,262797249.0,104.0,6.9,2413.0
2,0,11.7129,0.0,101.0,6.5,92.0
3,16000000,3.85949,81452156.0,127.0,6.1,34.0
4,0,8.38752,76578911.0,106.0,5.7,173.0


In [74]:
scaler = StandardScaler()
movies_standardized = scaler.fit_transform(movies.values)

In [75]:
movies.columns

Index(['budget', 'popularity', 'revenue', 'runtime', 'vote_average',
       'vote_count'],
      dtype='object')

In [76]:
{'eps': 10.0,
 'min_samples': 4,
 'distance_metric': 'euclidean',
 'num_labels': 2,
 'silhouette': 0.9428480074887184}

{'distance_metric': 'euclidean',
 'eps': 10.0,
 'min_samples': 4,
 'num_labels': 2,
 'silhouette': 0.9428480074887184}

In [77]:
eps = [0.1, 1.0, 2.0, 3.5, 5.0, 10.0]
min_samples = [4, 9, 15, 30, 50]
distance_metric = ['euclidean', 'manhattan']

dbscan_basket = []
best_param, best_score, best_labels = None, -1., []

In [ ]:
for e in eps:
    for ms in min_samples:
        for dm in distance_metric:
            dbscan = DBSCAN()
            labels = dbscan.fit(movies_standardized,e,ms,dm)
            num_labels = np.unique(labels).shape[0]
            if num_labels < 2: continue
            sil = silhouette_score(X, labels)
            params = {
                'eps': e,
                'min_samples': ms,
                'distance_metric': dm,
                'num_labels': num_labels,
                'silhouette': sil
            }

            dbscan_basket.append(params)
            if sil > best_score:
                best_param = params
                best_score = sil
                best_labels = labels

In [78]:
dbscan = DBSCAN()
labels=dbscan.fit(movies_standardized, 5,1,'euclidean')

labels

{0: 1,
 1: 1,
 2: 2,
 3: 2,
 4: 2,
 5: 2,
 6: 2,
 7: 2,
 8: 2,
 9: 2,
 10: 2,
 11: 2,
 12: 2,
 13: 2,
 14: 2,
 15: 2,
 16: 2,
 17: 2,
 18: 2,
 19: 2,
 20: 2,
 21: 2,
 22: 2,
 23: 2,
 24: 2,
 25: 2,
 26: 2,
 27: 2,
 28: 2,
 29: 2,
 30: 1,
 31: 2,
 32: 2,
 33: 2,
 34: 2,
 35: 2,
 36: 2,
 37: 2,
 38: 2,
 39: 2,
 40: 1,
 41: 1,
 42: 1,
 43: 2,
 44: 2,
 45: 2,
 46: 2,
 47: 2,
 48: 2,
 49: 2,
 50: 2,
 51: 2,
 52: 2,
 53: 2,
 54: 2,
 55: 2,
 56: 2,
 57: 2,
 58: 2,
 59: 2,
 60: 2,
 61: 2,
 62: 2,
 63: 2,
 64: 2,
 65: 2,
 66: 2,
 67: 2,
 68: 2,
 69: 2,
 70: 2,
 71: 2,
 72: 2,
 73: 2,
 74: 2,
 75: 2,
 76: 1,
 77: 1,
 78: 2,
 79: 2,
 80: 2,
 81: 2,
 82: 2,
 83: 2,
 84: 2,
 85: 2,
 86: 2,
 87: 2,
 88: 2,
 89: 2,
 90: 2,
 91: 1,
 92: 2,
 93: 2,
 94: 2,
 95: 2,
 96: 2,
 97: 2,
 98: 2,
 99: 2,
 100: 2,
 101: 2,
 102: 2,
 103: 2,
 104: 1,
 105: 2,
 106: 2,
 107: 2,
 108: 2,
 109: 2,
 110: 2,
 111: 2,
 112: 2,
 113: 2,
 114: 2,
 115: 2,
 116: 2,
 117: 2,
 118: 2,
 119: 2,
 120: 2,
 121: 2,
 122: 2,
 12

In [ ]:
test = RangeQuery(movies_standardized, 'manhattan', movies_standardized[2], 2)


In [ ]:
best_param